In [11]:
import streamlit as st
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px
import folium
from streamlit_folium import folium_static
from folium import IFrame

# chemin d'accès des fichiers de données
DATA_PATH = 'https://github.com/dataforgoodfr/12_zero_dechet_sauvage/raw/2-nettoyage-et-augmentation-des-donn%C3%A9es/Exploration_visualisation/data/data_zds_enriched.csv'
CORRECTION = 'https://github.com/dataforgoodfr/12_zero_dechet_sauvage/raw/1-exploration-des-donn%C3%A9es/Exploration_visualisation/data/releves_corrects_surf_lineaire.xlsx'
DATA_SPOT = 'https://github.com/dataforgoodfr/12_zero_dechet_sauvage/raw/1-exploration-des-donn%C3%A9es/Exploration_visualisation/data/export_structures_29022024.xlsx'

# fichier geojson pour localisation folium
FRANCE_DPTS = "departements-version-simplifiee.geojson"
FRANCE_REGIONS = "regions-avec-outre-mer.geojson"

# importation des données
data = pd.read_csv(DATA_PATH)
correction = pd.read_excel('releves_corrects_surf_lineaire.xlsx')
correction = correction.rename(columns={'DEF_\nSURFACE': 'DEF_SURFACE'})
spot = pd.read_excel('export_spots_02042024.xlsx')

# fusion et correction
data_corect = pd.merge(data, correction, on='ID_RELEVE', how='left')
data = data_corect[data_corect['DEF_SURFACE'] == 'OUI']

# Calcul de la densité
data['DENSITE'] = data['VOLUME_TOTAL']/data['SURFACE']
data = data[data['DENSITE'] < 20] # suppression d'une ligne avec une valeur de densité aberrante

data['DENSITE'] = data['DENSITE'].round(3)
data['SURFACE_ROND'] = data['SURFACE'].round(2)

couleur =  {
        'Littoral (terrestre)': 'lightblue',
        'Mer - Océan': 'darkblue',
        'Cours d\'eau': 'cyan',
        'Zone naturelle ou rurale (hors littoral et montagne)': 'green',
        'Zone urbaine': 'orange',
        'Lagune et étang côtier': 'red',
        'Multi-lieux': 'pink',
        'Montagne': 'grey',
        'Présent au sol (abandonné)': 'black'}

def couleur_milieu(type):
    return couleur.get(type, 'white')

C:\Users\diazj\AppData\Local\Temp\ipykernel_6000\4266156166.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DENSITE'] = data['VOLUME_TOTAL']/data['SURFACE']


In [ ]:
group0 = folium.FeatureGroup(name='<span style=\\"color: red;\\">red circles</span>')
for lat, lng in zip(range(500, 520), range(50,70)):
    folium.CircleMarker((lat/10, lng/10), color='red', radius=2).add_to(group0)
group0.add_to(m)

group1 = folium.FeatureGroup(name='<span style=\\"color: blue;\\">blue circles</span>')
for lat, lng in zip(range(500,520), range(70,50,-1)):
    folium.CircleMarker((lat/10, lng/10), color='blue', radius=2).add_to(group1)
group1.add_to(m)

In [30]:
gdf = gpd.read_file(FRANCE_REGIONS)

m = folium.Map(location=[data['LIEU_COORD_GPS_Y'].mean(), data['LIEU_COORD_GPS_X'].mean()])

for index, row in data.iterrows():

    popup_html = f"""
    <div style="width: 300px; height: 130px;">
        <h3>Densité: {row['DENSITE']} L/m²</h2>
        <h4>Volume total : {row['VOLUME_TOTAL']} litres</h2>
        <h4>Surface total : {row['SURFACE_ROND']} m²</h2>
        <h4>Type de milieu : {row['TYPE_MILIEU']}</h2>
    </div>
    """
    
    color = couleur_milieu(row['TYPE_MILIEU'])
    
    folium.CircleMarker(
        location=[row['LIEU_COORD_GPS_Y'], row['LIEU_COORD_GPS_X']],
        radius=np.log(row['DENSITE'] + 1)*15,  
        popup=folium.Popup(popup_html, max_width=300),
        color=color,
        fill=True, 
).add_to(m)




In [32]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Type de milieu</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:lightblue;opacity:0.7;'></span>Littoral (terrestre)</li>
    <li><span style='background:darkblue;opacity:0.7;'></span>Mer - Océan</li>
    <li><span style='background:cyan;opacity:0.7;'></span>Cours d'eau</li>
    <li><span style='background:green;opacity:0.7;'></span>Zone naturelle ou rurale (hors littoral et montagne)</li>
    <li><span style='background:orange;opacity:0.7;'></span>Zone urbaine</li>
    <li><span style='background:red;opacity:0.7;'></span>Lagune et étang côtier</li>
    <li><span style='background:pink;opacity:0.7;'></span>Multi-lieux</li>
    <li><span style='background:grey;opacity:0.7;'></span>Montagne</li>
    <li><span style='background:black;opacity:0.7;'></span>Présent au sol (abandonné)</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

m